In [1]:
## 코스피 주가 예측

## 코스피 데이터를 바탕으로 코스피 지수를 가장 정확하게 예측하는 모델을 만드시는 것이 목적.
# 모델 제한 없음, 전처리 방법 제한 없음.
# train_test_split test_size=0.3, random_state=42 test data 기준 RMSE 가장 낮은 모델을 만드시는 분이 우승.


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score


df = pd.read_csv('./data/kospi.csv', encoding ='cp949')

# df.info()
df.head()

# df.isnull().sum()

# df.dropna(inplace=True)



,전경련BSI,주택매매가격,건설BSI(전망),"실업률(계절조정,%)",어음부도율,일별수출액YoY,내국인출국자수YoY,코스피 200 변동성지수,통안채 364일,공사채 AAA 3년,...,재화수출,내수,GDP디플레이터,총저축률,총투자율,Reuter CRB(EW)상품선물지수,Reuter CRB(EW)에너지지수,Reuter CRB(EW)산업지수,Reuter CRB(EW)귀금속지수,코스피지수
0,96.4,70.13,83.3,3.7,0.09,14.50,-37.03,25.50,4.06,4.97,...,1.6,-0.4,3.1,32.8,32.3,243.7,247.7,217.5,322.4,633.4
1,90.3,70.83,86.7,3.7,0.08,10.56,-9.67,20.83,4.24,4.83,...,7.8,-0.4,4.0,33.3,32.3,247.6,255.7,247.3,326.4,669.9
2,91.4,70.57,86.7,3.8,0.06,14.83,0.93,26.98,4.60,4.43,...,7.8,0.3,4.0,33.3,31.8,249.9,268.7,232.3,340.1,713.5
3,109.6,69.99,92.4,3.9,0.06,24.13,2.85,28.33,4.66,4.74,...,7.8,0.3,4.0,33.3,31.8,255.3,283.3,256.6,364.1,759.5
4,110.3,69.70,64.1,3.8,0.08,25.49,14.34,25.35,4.51,5.21,...,10.9,0.3,3.3,34.7,31.8,262.6,292.7,262.0,368.3,697.5


In [5]:
y=df['코스피지수']
X=df.drop(columns=['코스피지수'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}, R2: {r2}')


MSE: 108.12799128058526, R2: 0.9424767666627333


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import Pipeline

# 데이터 로드
df = pd.read_csv('./data/kospi.csv', encoding='cp949')

# 결측치 처리
df = df.dropna()

# 특성과 타겟 분리
y = df['코스피지수']
X = df.drop(columns=['코스피지수'])

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 파이프라인 생성
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', XGBRegressor(random_state=42))
])

# 하이퍼파라미터 그리드 설정
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [3, 4, 5, 6],
    'regressor__learning_rate': [0.01, 0.1, 0.3],
    'regressor__subsample': [0.8, 0.9, 1.0],
    'regressor__colsample_bytree': [0.8, 0.9, 1.0],
}

# GridSearchCV를 사용한 하이퍼파라미터 튜닝
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 최적의 모델로 예측
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# RMSE 계산
rmse = root_mean_squared_error(y_test, y_pred)

print(f"Best parameters: {grid_search.best_params_}")
print(f"RMSE: {rmse}")



Best parameters: {'regressor__colsample_bytree': 0.8, 'regressor__learning_rate': 0.1, 'regressor__max_depth': 4, 'regressor__n_estimators': 300, 'regressor__subsample': 0.8}
RMSE: 107.53294557562116


In [2]:
# 특성 중요도 출력
feature_importance = best_model.named_steps['regressor'].feature_importances_
feature_importance_df = pd.DataFrame({'feature': X.columns, 'importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)
print("\nTop 10 important features:")
print(feature_importance_df.head(10))


Top 10 important features:
                 feature  importance
28   Reuter CRB(EW)귀금속지수    0.581957
25  Reuter CRB(EW)상품선물지수    0.134833
12        국고채10년 - 통안채1년    0.076796
1                 주택매매가격    0.058720
23                  총저축률    0.027841
27    Reuter CRB(EW)산업지수    0.022888
22              GDP디플레이터    0.019648
11            국고채5년 - CD    0.011858
16              한국(OECD)    0.008810
5               일별수출액YoY    0.008465


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import Pipeline

# 데이터 로드
df = pd.read_csv('./data/kospi.csv', encoding='cp949')

# 결측치 처리
df = df.dropna()

# 특성과 타겟 분리
y = df['코스피지수']
X = df.drop(columns=['코스피지수'])

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 특성 엔지니어링 추가
polynomial_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = polynomial_features.fit_transform(X)
X_train_poly, X_test_poly, y_train, y_test = train_test_split(X_poly, y, test_size=0.3, random_state=42)

# 앙상블 모델 생성
xgb = XGBRegressor(random_state=42)
rf = RandomForestRegressor(random_state=42)
gb = GradientBoostingRegressor(random_state=42)

ensemble = VotingRegressor([
    ('xgb', xgb),
    ('rf', rf),
    ('gb', gb)
])

# 파이프라인 생성
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', ensemble)
])

# 하이퍼파라미터 그리드 설정 
param_grid = {
    'regressor__xgb__n_estimators': [100, 300, 500],
    'regressor__xgb__max_depth': [3, 5, 7],
    'regressor__xgb__learning_rate': [0.01, 0.1, 0.3],
    'regressor__rf__n_estimators': [100, 300, 500],
    'regressor__rf__max_depth': [5, 10, None],
    'regressor__gb__n_estimators': [100, 300, 500],
    'regressor__gb__max_depth': [3, 5, 7],
    'regressor__gb__learning_rate': [0.01, 0.1, 0.3],
}

# GridSearchCV를 사용한 하이퍼파라미터 튜닝
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_poly, y_train)

# 최적의 모델로 예측
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_poly)

# RMSE 계산
rmse = root_mean_squared_error(y_test, y_pred)

print(f"Best parameters: {grid_search.best_params_}")
print(f"RMSE: {rmse}")

# 특성 중요도 출력 (앙상블 모델의 경우 개별 모델의 특성 중요도를 평균화)
feature_importance = np.mean([
    best_model.named_steps['regressor'].estimators_[0].feature_importances_,
    best_model.named_steps['regressor'].estimators_[1].feature_importances_,
    best_model.named_steps['regressor'].estimators_[2].feature_importances_
], axis=0)

feature_names = polynomial_features.get_feature_names_out(X.columns)
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)
print("\nTop 10 important features:")
print(feature_importance_df.head(10))

KeyboardInterrupt: 